# Artificial Intelligence - Fall 2020 - Laboratory 07

## _Constraint Satisfaction Problems (CSPs)_

c: Alexandra Dobrescu <alexandramaria.digital@gmail.com>

## Introduction

**Constraint Satisfaction Problems** are defined by:
* a collection of discrete **variables**;
* a **domain (range) of values** for each variable;
* a set of **constraints** over variables.

## From definitions to know-how

In [1]:
# Useful libraries:
from copy import copy, deepcopy
from itertools import combinations

Pythonic speaking:
  - each variable will be represented by a string,
  
```python
    vars_list = ['A', 'B', 'C']
```

   - the set of domains will be a dictionary with one entry for each variable:
     + the key will be the name of the variable,
     + the value is given by the range values of that variable

```python
    domains_dict = {'A': [1, 2, 3], 'B': [1, 5, 9], 'C': [-2, -1]}
```

   - a constraint will be represented by a `tuple` consisting of:
     + a `list` of variables involved in the constraint
     + an anonymous function that returns `True` or `False`

```python
    constraints = [(['A', 'B', 'C'], lambda a, b, c: a + b + c == 0)]
```

The **solution** will be represented through a dictionary that indicates a value for each variable (e.g. `{'A': 1,'B': 1,'C': -2}`).

The **cost** is given by the number of constraints ignored by the solution.

Math Problem

In [2]:
VarsA = ["A", "B", "C", "D", "E"]
DomainsA = {v: [i for i in range(10)] for v in VarsA}
ConstraintsA = [(list(p), lambda x,y: x != y) for p in combinations(VarsA, 2)] # all the values are different
ConstraintsA.append((["A","B"], lambda a, b: a + b == 10))
ConstraintsA.append((["B","D"], lambda b, d: b + d == 6))
ConstraintsA.append((["C"], lambda c: c < 5))
ConstraintsA.append((["A"], lambda a: a > 5))
ConstraintsA.append((["A","B","C","D","E"], lambda a, b, c, d, e: a + b + c + d + e == 30))
MathProblem = {"Vars": VarsA, "Domains": DomainsA, "Constraints": ConstraintsA}

Map-Coloring Problem

In [3]:
VarsC = ["France", "Germany", "Loux", "Belgium", "Netherlands"]
DomainsC = {v: ["blue", "red", "yellow", "green"] for v in VarsC}
ConstraintsC = []
for (a, b) in [("France", "Germany"), ("France", "Belgium"), ("France", "Loux"),
               ("Belgium", "Netherlands"), ("Belgium", "Loux"), ("Belgium", "Germany"),
               ("Loux", "Germany"), ("Netherlands", "Germany")]:
    ConstraintsC.append(([a, b], lambda a, b: a != b))
ColoringProblem = {"Vars": VarsC, "Domains": DomainsC, "Constraints": ConstraintsC}

### Task 0

Implement `get_constraints` function which receives a variable `var` and a list of constraints `constraints` and return only those constraints that entail the given variable.

_Expected output:_

```
Constraints = [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
get_constraints("B", Constraints)
==> [(["A", "B"], lambda a,b: a<b)]
get_constraints("A", Constraints)
==> [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
```

In [4]:
def get_constraints(var, constraints):
    # TODO
    spec_constr = []
    for val in constraints:
        (vars, constr) = val
        if var in vars:
            spec_constr.append(val)
    return spec_constr

get_constraints("France", ConstraintsC) # => [(['France', 'Germany'], ...), (['France', 'Belgium'], ...), (['France', 'Loux'], ...)]

[(['France', 'Germany'], <function __main__.<lambda>(a, b)>),
 (['France', 'Belgium'], <function __main__.<lambda>(a, b)>),
 (['France', 'Loux'], <function __main__.<lambda>(a, b)>)]

### Task 1

Implement the `fixed_constraints` function which receives `solution` as a partial solution and a set of constraints `constraints`. The function returns only those constraints that can be evaluated (i.e. the variables involved are found in the partial solution).

In [5]:
def fixed_constraints(solution, constraints):
    # TODO
    verifiable_constr = []
    for val in constraints:
        (vars,constr) = val
        check = True
        for x in vars:
            if x not in solution.keys():
                check = False
        if check==True:
            verifiable_constr.append((vars,constr))
    return verifiable_constr

print(fixed_constraints({"France": "blue", "Belgium": "green"}, ConstraintsC)) # => [(['France', 'Belgium'], ...)]
print(fixed_constraints({"A": "1", "C": "2"}, ConstraintsA)) # => [(['A', 'C'], ...), (['C'], ...), (['A'], ...)]

[(['France', 'Belgium'], <function <lambda> at 0x000002526C610D30>)]
[(['A', 'C'], <function <listcomp>.<lambda> at 0x000002526C6101F0>), (['C'], <function <lambda> at 0x000002526C610790>), (['A'], <function <lambda> at 0x000002526C610820>)]


### Task 2

Implement the `check_constraint` function which receives as a constraint the variable `constraint`, a partial solution `solution` and returns `True` if the solution respects the constraint and `False` otherwise.

_**Hint:**_ to call a function knowing the list of its parameters use `f(*args)`, where `args` is the list of arguments.

In [6]:
def check_constraint(solution, constraint):
    # TODO
    (vars, constr) = constraint
    vals = []
    for i in vars:
        vals.append(solution[i])
    return constr(*vals)

print(check_constraint({"France": "blue", "Belgium": "green"}, ConstraintsC[1])) # => True
print(check_constraint({"France": "blue", "Belgium": "blue"}, ConstraintsC[1])) # => False
print(check_constraint({"C": 10, "A": 10}, ConstraintsA[-2])) # => True
print(check_constraint({"C": 10, "A": 3}, ConstraintsA[-2])) # => False

True
False
True
False


### Task 3: PCSP algorithm

Fill in what is missing in the PCSP algorithm.

* `vars` - variables that remain to be checked
* `domains` - the domains for the variables that remain to be verified, with the values that remain to be verified for each variable
* `constrains` - the list of constraints
* `acceptable_cost` - the cost for which the solution is considered satisfactory
* `solution` - the partial solution, containing values for the variables checked so far
* `cost` - the cost of unsatisfied constraints

Function return value is `True` if a satisfactory complete solution has been found (see acceptable cost), and `False` otherwise.

Two global variables are used:

* `best_cost` - the best (smallest) cost so far in exploration, for a complete solution
* `best_solution` - the solution corresponding to the best cost

In [7]:
def PCSP(vars, domains, constraints, acceptable_cost, solution, cost):
    global best_solution
    global best_cost
    if not vars:
        # If there are no more variables, we have reached a better solution
        print("New best: " + str(cost) + " - " + str(solution))
        # TODO: save the newly discovered solution
        best_solution = solution
        best_cost = cost
        # TODO: if it is good enough, the function returns True
        if best_cost < acceptable_cost:
            return True
    
    elif not domains[vars[0]]:
        # If there are no more values in the domains, the searching process ends
        return False
    elif cost == best_cost:
        # If we have already reached a cost identical with the one of the best solution, we will not go any further
        return False
    else:
        # TODO: Choose the first variable and the first value in the domains
        var = vars[0]
        val = domains[var].pop(0)

        # TODO: Build the new solution
        new_solution = deepcopy(solution)
        new_solution[var] = val

        # TODO: We get the list of constraints that can be evaluated
        constr_eval = get_constraints(var,constraints)
        constr_fixed = fixed_constraints(new_solution,constr_eval)
        # TODO: We compute the cost of the new partial solution (each constraint ignored = 1)
        new_cost = cost
        for con in constr_fixed:
            if not check_constraint(new_solution,con):
                new_cost += 1

        # We check if the new cost is lower than the best cost
        if new_cost < best_cost:
            # TODO:
            # If the new cost is lower than the best known, we solve for the rest of the variables
            # If the recursive call returns True, a good enough solution has been found, so we return Trunew_vars = []
            new_vars = deepcopy(vars[1:])
            var_domain = domains.pop(var)
            new_domains = deepcopy(domains)
            
            next_step = PCSP(new_vars,new_domains,constraints, acceptable_cost,new_solution,new_cost)
            if next_step:
                return True
            
            domains[var] = var_domain
        
        # Check for the rest of the values
        # TODO:
        PCSP(vars,deepcopy(domains),constraints,acceptable_cost,solution,cost)
    
# A wrapper that instantiates global variables
def run_pcsp(problem, acceptable_cost):
    global best_solution
    global best_cost

    [vars, domains, constraints] = [problem[e] for e in ["Vars", "Domains", "Constraints"]]
    
    best_solution = {}
    best_cost = len(constraints)

    if PCSP(vars, deepcopy(domains), constraints, acceptable_cost, {}, 0):
        print("Best found: " + str(best_cost) + " - " + str(best_solution))
    else:
        print("Acceptable solution not found; " + "Best found: " + str(best_cost) + " - " + str(best_solution))
        
# Run the algorithm
run_pcsp(MathProblem, 1)
run_pcsp(ColoringProblem, 1)

New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Acceptable solution not found; Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
New best: 6 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red'}
New best: 4 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue'}
New best: 3 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow'}
New best: 2 - {'France': 'blue'

Expected output for numbers:

```
New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
```

Expected output for country colors:

```
New best:  8  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  6  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
New best:  4  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  3  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  2  -  {'Loux': 'red', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
Best found: 1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
```

Feedback

Hello! 

Thank you so much for including more code documentation. This lab was much more clear for me and I had an easier time solving it. I feel really good about being able to get close to the good solution, although I didn't manage to obtain exaclty the right output. Do you think you could take a look at my code and see what I'm doing wrong?

Either way, you're doing great for someone who is teaching for the first time. I really enjoy your lab :D

See you next week!

Irina/Mona